<a href="https://colab.research.google.com/github/kushc2004/LLM/blob/main/Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

In [3]:
# Install necessary packages
!pip install transformers

# Import necessary libraries
import json
import traceback
from typing import Dict, List, Optional
from transformers import pipeline

# Define the Agent class
class Agent:
    def __init__(self, name, description, client, llm="llama2", **kwargs):
        self.name = name
        self.description = description
        self.client = client
        self.system_prompt = "You're a helpful assistant."
        self.kwargs = kwargs
        self.llm = llm

    def llm_call(self, prompt: Optional[str] = None, messages: Optional[List] = None, seed: int = 10) -> str:
        generator = pipeline('text-generation', model='meta-llama/Llama-2-7b-hf')
        if prompt:
            response = generator(prompt, max_length=500, num_return_sequences=1)
            return response[0]['generated_text']
        return ""

    def generate_reply(self, task: str, state: Dict, sender: "Agent") -> (str, Dict):
        return ("This is a reply from the agent. REPLY NOT IMPLEMENTED! Terminate the whole process!", state)

# Define the Formulator class
class Formulator(Agent):
    def __init__(self, client, **kwargs):
        super().__init__(
            name="Formulator",
            description="This is a mathematical formulator agent that is an expert in mathematical and optimization modeling and can define and modify variables, constraints, and objective functions.",
            client=client,
            **kwargs,
        )

    def _formulate(self, target_type: str, state):
        for target in state[target_type]:
            if target["status"] == "not_formulated":
                self._create_new_formulation(target, target_type, state)
            elif target["status"] == "runtime_error":
                self._fix_available_formulation(target, target_type, state)
            elif target["status"] == "formulated":
                continue
            else:
                error_msg = f"Invalid status: {json.dumps(target, indent=4)}"
                raise RuntimeError(error_msg)
        return

    def _create_new_formulation(self, target, target_type: str, state):
        print(f"Formulating {target_type} ...")
        context = {}
        context[target_type] = {}
        prompt_template = """
        You are an expert mathematical formulator and an optimization professor at a top university. Your task is to model {targetType} of the problem in the standard LP or MILP form.

        Here is a {targetType} we need you to model:
        -----
        {targetDescription}
        -----

        Here is some context on the problem:
        -----
        {background}
        -----

        Here is the list of available variables:
        -----
        {variables}
        -----

        And finally, here is list of input parameters:
        -----
        {parameters}
        -----

        First, take a deep breath and explain how we should define the {targetType}. Feel free to define new variables if you think it is necessary. Then, generate a json file accordingly with the following format (STICK TO THIS FORMAT!):

        {{
            "{targetType}": {{
              "description": "The description of the {targetType}",
              "formulation": "The LaTeX mathematical expression representing the formulation of the {targetType}"
            }},
            "auxiliary_constraints": [
                {{
                    "description": "The description of the auxiliary constraint",
                    "formulation": "The LaTeX mathematical expression representing the formulation of the auxiliary constraint"
                }}
            ],
            "new_variables": [
                {{
                    "definition": "The definition of the variable",
                    "symbol": "The symbol for the variable",
                    "shape": [ "symbol1", "symbol2", ... ]
                }}
            ]
        }}

        - Your formulation should be in LaTeX mathematical format (do not include the $ symbols).
        - Note that I'm going to use python json.loads() function to parse the json file, so please make sure the format is correct (don't add ',' before enclosing '}}' or ']' characters.
        - Generate the complete json file and don't omit anything.
        - Use '```json' and '```' to enclose the json file.
        - Important: You can not define new parameters. You can only define new variables.Use CamelCase and full words for new variable symbols, and do not include indices in the symbol (e.g. ItemsSold instead of itemsSold or items_sold or ItemsSold_i)
        - Use \\textup{{}} when writing variable and parameter names. For example (\\sum_{{i=1}}^{{N}} \\textup{{ItemsSold}}_{{i}} instead of \\sum_{{i=1}}^{{N}} ItemsSold_{{i}})
        - Use \\quad for spaces.
        - Use empty list ([]) if no new variables are defined.
        - Always use non-strict inequalities (e.g. \\leq instead of <), even if the constraint is strict.
        - Define auxiliary constraints when necessary. Set it to an empty list ([]) if no auxiliary constraints are needed. If new auxiliary constraints need new variables, add them to the "new_variables" list too.

        Take a deep breath and solve the problem step by step.
        """
        prompt = prompt_template.format(
            background=state["background"],
            targetType=target_type,
            targetDescription=target["description"],
            variables=json.dumps(
                [
                    {
                        "definition": v["definition"],
                        "symbol": v["symbol"],
                        "shape": v["shape"],
                    }
                    for v in state["variables"]
                ],
                indent=4,
            ),
            parameters=json.dumps(
                [
                    {
                        "definition": p["definition"],
                        "symbol": p["symbol"],
                        "shape": p["shape"],
                    }
                    for p in state["parameters"]
                ],
                indent=4,
            ),
        )

        cnt = 3
        while cnt > 0:
            cnt -= 1
            try:
                response = self.llm_call(prompt=prompt, seed=cnt)
                print("=" * 10)
                print(response)
                print("=" * 10)
                output = response
                if "```json" in output:
                    output = output[output.find("```json") + 7 :]
                    output = output[: output.rfind("```")]

                while output[-1] != "}":
                    output = output[:-1]

                while output[0] != "{":
                    output = output[1:]

                if "$" in output:
                    output = output.replace("$", "")

                formulation_start = output.find('"formulation"')
                auxiliary_constraints_start = output.find('"auxiliary_constraints"')
                while output[auxiliary_constraints_start] != "}":
                    auxiliary_constraints_start -= 1
                while output[auxiliary_constraints_start] != '"':
                    auxiliary_constraints_start -= 1

                formulation = output[
                    formulation_start + 16 : auxiliary_constraints_start
                ]
                output = (
                    output[: formulation_start + 16]
                    + output[auxiliary_constraints_start:]
                )

                formulation = formulation.replace("\\\\", "\\")
                output = output.replace("\\", "\\\\")
                output = output.replace("\\", "\\\\")
                output = output.replace("\\\\quad", "\\\\")

                update = json.loads(output)
                update[target_type]["formulation"] = formulation

                related_stuff = self.get_related_stuff(
                    state, formulation, update["new_variables"]
                )
                update["variables_mentioned"] = related_stuff["variables_mentioned"]
                update["parameters_mentioned"] = related_stuff["parameters_mentioned"]

                if not "new_variables" in update.keys():
                    raise Exception("new_variables is not in the json file!")
                if not "formulation" in update[target_type].keys():
                    raise Exception("formulation is not in the json file!")
                if not "auxiliary_constraints" in update.keys():
                    update["auxiliary_constraints"] = []

                break
            except Exception as e:
                print(traceback.format_exc())
                print("=" * 10)
                print(e)
                print("=" * 10)
                print(prompt)
                print("=" * 10)
                print(response)
                print("=" * 10)
                print(
                    f"Invalid json format in {target_type} formulation!\n{e}\n Try again ..."
                )

        if cnt == 0:
            raise Exception("Invalid json format!")

        all_variable_symbols = [variable["symbol"] for variable in state["variables"]]

        for variable in update["new_variables"]:
            if variable["symbol"] in all_variable_symbols:
                continue
            else:
                variable["status"] = "formulated"
                state["variables"].append(variable)
                if not variable["symbol"] in update["variables_mentioned"]:
                    update["variables_mentioned"].append(variable["symbol"])

        target["formulation"] = update[target_type]["formulation"]
        target["status"] = "formulated"

        target["related_variables"] = update["variables_mentioned"]
        target["related_parameters"] = update["parameters_mentioned"]

        if "auxiliary_constraints" in update.keys():
            for constraint in update["auxiliary_constraints"]:
                constraint["status"] = "formulated"

                constraint["formulation"] = (
                    constraint["formulation"]
                    .replace("\\\\", "\\")
                    .replace("\\\\", "\\")
                )

                related_stuff = self.get_related_stuff(
                    state, constraint["formulation"], []
                )
                constraint["related_variables"] = related_stuff["variables_mentioned"]
                constraint["related_parameters"] = related_stuff["parameters_mentioned"]

                state["constraint"].append(constraint)

        print(f"Formulation: {target['formulation']}")
        print("---")
        return

    def _fix_available_formulation(self, target, target_type: str, state):
        print(f"Fixing {target_type} ...")
        context = {}
        context[target_type] = {}
        context[target_type]["description"] = target["description"]

        context["variables"] = state["variables"]
        context["parameters"] = state["parameters"]
        context["formulation"] = target["formulation"]
        context["error"] = state["error_message"]

        fix_prompt_template = """
        You are a mathematical formulator working with a team of optimization experts. The objective is to tackle a complex optimization problem, and your role is to fix a previously modelled {target}.

        Recall that the {target} you modelled was

        -----
        {constraint}
        -----

        and your formulation you provided was

        -----
        {formulation}
        -----

        The error message is

        -----
        {error}
        -----

        Here are the variables you have so far defined:

        -----
        {variables}
        -----

        Here are the parameters of the problem

        -----
        {parameters}
        -----

        Your task is carefully inspect the old {target} and fix it when you find it actually wrong.
        After fixing it modify the formulation. Please return the fixed JSON string for the formulation.

        The current JSON is

        -----
        {json}
        -----

        """
        prompt = fix_prompt_template.format(
            target=target_type,
            constraint=json.dumps(context[target_type]["description"], indent=4),
            variables=json.dumps(context["variables"], indent=4),
            parameters=json.dumps(context["parameters"], indent=4),
            formulation=json.dumps(context["formulation"], indent=4),
            json=json.dumps(target),
            error=context["error"],
        )

        cnt = 3
        while cnt > 0:
            cnt -= 1
            try:
                output = self.llm_call(prompt=prompt)
                output = output[output.find("```json") + 7 :]
                output = output[: output.rfind("```")]

                output = output.replace(" \\", " \\\\")
                update = json.loads(output)

                break
            except Exception as e:
                print(e)
                print(
                    f"Invalid json format in {target_type} formulation! Try again ..."
                )

        if cnt == 0:
            raise Exception("Invalid json format!")

        target["formulation"] = update["formulation"]
        target["related_variables"] = update["related_variables"]
        target["related_parameters"] = update["related_parameters"]
        target["status"] = "formulated"

        return

    def generate_reply(self, task: str, state: Dict, sender: Agent) -> (str, Dict):
        print("- Formulator agent is called!")
        print()

        self._formulate("constraint", state)
        self._formulate("objective", state)

        return "Formulation Done! Now we can write the code.", state

    def get_related_stuff(self, state, formulation, new_variables):
        ret = {}
        ret["variables_mentioned"] = []
        ret["parameters_mentioned"] = []

        symbols_mentioned = []
        for i in range(len(formulation)):
            if formulation[i : i + 8] == "\\textup{":
                j = i + 8
                while formulation[j] != "}":
                    j += 1
                symbols_mentioned.append(formulation[i + 8 : j])

        all_parameter_symbols = [
            parameter["symbol"] for parameter in state["parameters"]
        ]
        all_variable_symbols = [variable["symbol"] for variable in state["variables"]]
        all_variable_symbols += [variable["symbol"] for variable in new_variables]

        for symbol in symbols_mentioned:
            if symbol in all_parameter_symbols:
                ret["parameters_mentioned"].append(symbol)
            elif symbol in all_variable_symbols:
                ret["variables_mentioned"].append(symbol)
            elif symbol.lower().strip() in [
                "min",
                "max",
                "subject to",
                "s.t.",
                "st",
                "minimize",
                "maximize",
                "sum",
                "for all",
                "forall",
                "such that",
                "and",
                "or",
                "if",
                "then",
                "else",
                "otherwise",
                "for each",
                "exists",
                "foreach",
            ]:
                continue
            elif " " in symbol:
                continue
            else:
                raise Exception(f"Symbol {symbol} is not defined!")

        return ret

# Define the Programmer class
class Programmer(Agent):
    def __init__(self, client, solver="gurobipy", debugger_on=True, **kwargs):
        super().__init__(
            name="Programmer",
            description="This is a mathematical programmer agent that is an expert in writing, modifying, and debugging code for optimization problems from the mathematical formulation of the problem.",
            client=client,
            **kwargs,
        )

        self._debugger_on = debugger_on
        self.solver = solver

    def generate_reply(self, task: str, state: Dict, sender: Agent) -> (str, Dict):
        print("- Programmer agent is called!")
        print()

        if state["solution_status"] == "runtime_error":
            return self._debug_code(state=state)
        elif state["solution_status"] is None:
            return self._generate_code_from_formulation(state=state)
        else:
            raise Exception(f"Invalid solver_output_status {state['solver_output_status']}!")

    def _debug_code(self, state: Dict) -> (str, Dict):
        if not self._debugger_on:
            raise Exception("Debugger is off. Execution failed")

        error_line = None
        bogus_context = None

        for target in ["constraint", "objective", "variables"]:
            for item in state[target]:
                if item["status"] == "runtime_error":
                    bogus_context = item

        context = {}
        prep_code = state["prep_code"]

        if "description" in bogus_context:
            error_line = bogus_context["code"]
            error_message = state["error_message"]
            for parameter in state["parameters"]:
                if parameter["symbol"] in bogus_context["related_parameters"]:
                    prep_code += parameter["code"] + "\n"

            for variable in state["variables"]:
                if variable["symbol"] in bogus_context["related_variables"]:
                    if not "code" in variable:
                        raise Exception(f"Variable {variable} is not coded yet!")

                    prep_code += variable["code"] + "\n"
            prompt = debugging_refined_template_target.format(
                target=target,
                prep_code=prep_code,
                error_line=error_line,
                error_message=error_message,
            )

        elif "definition" in bogus_context:
            error_line = bogus_context["code"]
            error_message = state["error_message"]

            prompt = debugging_refined_template_variable.format(
                target=target,
                prep_code=prep_code,
                error_line=error_line,
                error_message=error_message,
            )

        else:
            raise Exception(f"Invalid bogus_context {bogus_context}! \n {json.dumps(state, indent=4)}")

        cnt = 3
        while cnt > 0:
            cnt -= 1
            try:
                output = self.llm_call(prompt=prompt, seed=cnt)
                output = output[output.find("```json") + 7 :]
                output = output[: output.rfind("```")]

                update = json.loads(output)

                if update["status"] == "correct":
                    bogus_context["status"] = "formulation_error"
                    return update["reason"], state
                elif update["status"] == "fixed":
                    bogus_context["status"] = "coded"
                    bogus_context["code"] = update["fixed_code"]
                    return "The code is fixed! Try evaluating it again.", state
                else:
                    raise Exception(f"Invalid status {update['status']}!")

            except Exception as e:
                print(e)
                print(f"Invalid json format {response}! Try again ...")

    def _generate_code_from_formulation(self, state: Dict) -> (str, Dict):
        for variable in state["variables"]:
            print(f"Programming variable {variable['symbol']} ...")

            if variable["status"] == "not_formulated":
                raise Exception(f"Variable {variable} is not formulated yet!")

            elif variable["status"] == "formulated":
                context = {}
                context["definition"] = variable["definition"]
                context["symbol"] = variable["symbol"]
                context["shape"] = variable["shape"]

                variable_definition_prompt_templates = [
                    """
                    You're an expert programmer in a team of optimization experts. The goal of the team is to solve an optimization problem. Your responsibility is to write {solver} code for defining variables of the problem.
                    """,
                    """
                    Here's a variable we need you to write the code for defining:

                    -----
                    {variable}
                    -----

                    Assume the parameters are defined. Now generate a code accordingly and enclose it between "=====" lines. Only generate the code, and don't generate any other text. Here's an example:

                    **input**:

                    {{
                        "definition": "Quantity of oil i bought """
                        ]

In [12]:
from typing import Dict
import json
from transformers import pipeline

test_code_template = """
You are in a team of optimization experts, and you are responsible for testing the validity of the solution
    obtained for the problem. Your job is to verify that all the constraints of the problem are satisfied by the
    solution. Here is one constraint you need to verify

    {CONSTRAINT}

    This constraint is built based on the following variables

    {VARIABLE}

    Now the problem is solved and solution is obtained.
    Please write Python code that gets values from the Python variables and then verify the constraint.
"""

class AutoTester(Agent):

    def __init__(self, client=None, solver="gurobipy", **kwargs):
        super(AutoTester, self).__init__(
            name="AutoTester",
            description="This is a testing agent that double checks the validity of the solution",
            client=client,
            **kwargs
        )
        self.solver = solver

    def llm_call(self, prompt: str) -> str:
        # Load model directly
        from transformers import AutoTokenizer, AutoModelForCausalLM
        tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
        model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf")
        response = generator(prompt, max_length=500, num_return_sequences=1)
        return response[0]['generated_text']

    def generate_reply(
        self,
        task: str,
        state: Dict,
        sender: "Agent",
    ) -> (str, Dict):

        print("- Testing agent is called! ")

        # Example of how to use the test_code_template
        constraint = "x + y <= 10"
        variable = "x, y"
        prompt = test_code_template.format(CONSTRAINT=constraint, VARIABLE=variable)

        response = self.llm_call(prompt)
        print(response)

        return response, state

# Example usage
if __name__ == "__main__":
    tester = AutoTester()
    task = "Verify the constraints"
    state = {}
    sender = None
    tester.generate_reply(task, state, sender)

- Testing agent is called! 


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-2-7b-hf.
403 Client Error. (Request ID: Root=1-6672b207-5f522bda7419d1531cfc87f3;1f3dc5cc-722e-4be8-94f2-bed00d67ace0)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-hf/resolve/main/config.json.
Access to model meta-llama/Llama-2-7b-hf is restricted and you are not in the authorized list. Visit https://huggingface.co/meta-llama/Llama-2-7b-hf to ask for access.

In [13]:
# Install necessary packages
!pip install transformers

# Import necessary libraries
from typing import Dict, Tuple
import json
from transformers import pipeline

# Define the test code template
test_code_template = """
You are in a team of optimization experts, and you are responsible for testing the validity of the solution
    obtained for the problem. Your job is to verify that all the constraints of the problem are satisfied by the
    solution. Here is one constraint you need to verify

    {CONSTRAINT}

    This constraint is built based on the following variables

    {VARIABLE}

    Now the problem is solved and solution is obtained.
    Please write Python code that gets values from the Python variables and then verify the constraint.
"""

# Define the Agent class
class Agent:
    def __init__(self, name, description, client, llm="gpt-2", **kwargs):
        self.name = name
        self.description = description
        self.client = client
        self.system_prompt = "You're a helpful assistant."
        self.kwargs = kwargs
        self.llm = llm

    def llm_call(self, prompt: str) -> str:
        generator = pipeline('text-generation', model=self.llm)
        response = generator(prompt, max_length=500, num_return_sequences=1)
        return response[0]['generated_text']

    def generate_reply(self, task: str, state: Dict, sender: "Agent") -> Tuple[str, Dict]:
        return ("This is a reply from the agent. REPLY NOT IMPLEMENTED! Terminate the whole process!", state)

# Define the AutoTester class
class AutoTester(Agent):
    def __init__(self, client=None, solver="gurobipy", **kwargs):
        super(AutoTester, self).__init__(
            name="AutoTester",
            description="This is a testing agent that double checks the validity of the solution",
            client=client,
            **kwargs
        )
        self.solver = solver

    def generate_reply(self, task: str, state: Dict, sender: "Agent") -> Tuple[str, Dict]:
        print("- Testing agent is called! ")

        # Example of how to use the test_code_template
        constraint = "x + y <= 10"
        variable = "x, y"
        prompt = test_code_template.format(CONSTRAINT=constraint, VARIABLE=variable)

        response = self.llm_call(prompt)
        print(response)

        return response, state

# Example usage
if __name__ == "__main__":
    tester = AutoTester()
    task = "Verify the constraints"
    state = {}
    sender = None
    tester.generate_reply(task, state, sender)

- Testing agent is called! 


OSError: gpt-2 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [1]:
# Install necessary packages
!pip install transformers

# Import necessary libraries
from typing import Dict, Tuple
import json
from transformers import pipeline

# Define the test code template
test_code_template = """
You are in a team of optimization experts, and you are responsible for testing the validity of the solution
    obtained for the problem. Your job is to verify that all the constraints of the problem are satisfied by the
    solution. Here is one constraint you need to verify

    {CONSTRAINT}

    This constraint is built based on the following variables

    {VARIABLE}

    Now the problem is solved and solution is obtained.
    Please write Python code that gets values from the Python variables and then verify the constraint.
"""

# Define the Agent class
class Agent:
    def __init__(self, name, description, client=None, llm="gpt2", **kwargs):
        self.name = name
        self.description = description
        self.client = client
        self.system_prompt = "You're a helpful assistant."
        self.kwargs = kwargs
        self.llm = llm

    def llm_call(self, prompt: str) -> str:
        generator = pipeline('text-generation', model=self.llm)
        response = generator(prompt, max_length=500, num_return_sequences=1)
        return response[0]['generated_text']

    def generate_reply(self, task: str, state: Dict, sender: "Agent") -> Tuple[str, Dict]:
        return ("This is a reply from the agent. REPLY NOT IMPLEMENTED! Terminate the whole process!", state)

# Define the AutoTester class
class AutoTester(Agent):
    def __init__(self, client=None, solver="gurobipy", **kwargs):
        super(AutoTester, self).__init__(
            name="AutoTester",
            description="This is a testing agent that double checks the validity of the solution",
            client=client,
            **kwargs
        )
        self.solver = solver

    def generate_reply(self, task: str, state: Dict, sender: "Agent") -> Tuple[str, Dict]:
        print("- Testing agent is called! ")

        # Example of how to use the test_code_template
        constraint = "x + y <= 10"
        variable = "x, y"
        prompt = test_code_template.format(CONSTRAINT=constraint, VARIABLE=variable)

        response = self.llm_call(prompt)
        print(response)

        return response, state

# Example usage
if __name__ == "__main__":
    tester = AutoTester()
    task = "Verify the constraints"
    state = {}
    sender = None
    tester.generate_reply(task, state, sender)

- Testing agent is called! 


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



You are in a team of optimization experts, and you are responsible for testing the validity of the solution
    obtained for the problem. Your job is to verify that all the constraints of the problem are satisfied by the 
    solution. Here is one constraint you need to verify 

    x + y <= 10

    This constraint is built based on the following variables 

    x, y

    Now the problem is solved and solution is obtained. 
    Please write Python code that gets values from the Python variables and then verify the constraint.



- 3 * k = 10

- 3 * k + 100

- 3 * k + 10000


Once the solution is determined, the value being checked is created. This will add one or two more constraints, or even more if the problem satisfies all the constraints for that solution.

For example if you wanted to test the validity of a given rule that adds two different conditions, you could find the following code in your Python program:

import matplotlib.pyplot2 import matplotlib.pyplot2.yimport pandoc

f

In [2]:
# Install necessary packages
!pip install transformers matplotlib pandas

# Import necessary libraries
from typing import Dict, Tuple
import json
from transformers import pipeline
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import sys

# Define the test code template
test_code_template = """
You are in a team of optimization experts, and you are responsible for testing the validity of the solution
    obtained for the problem. Your job is to verify that all the constraints of the problem are satisfied by the
    solution. Here is one constraint you need to verify

    {CONSTRAINT}

    This constraint is built based on the following variables

    {VARIABLE}

    Now the problem is solved and solution is obtained.
    Please write Python code that gets values from the Python variables and then verify the constraint.
"""

# Define the Agent class
class Agent:
    def __init__(self, name, description, client=None, llm="gpt2", **kwargs):
        self.name = name
        self.description = description
        self.client = client
        self.system_prompt = "You're a helpful assistant."
        self.kwargs = kwargs
        self.llm = llm

    def llm_call(self, prompt: str) -> str:
        generator = pipeline('text-generation', model=self.llm)
        response = generator(prompt, max_length=500, num_return_sequences=1)
        return response[0]['generated_text']

    def generate_reply(self, task: str, state: Dict, sender: "Agent") -> Tuple[str, Dict]:
        return ("This is a reply from the agent. REPLY NOT IMPLEMENTED! Terminate the whole process!", state)

# Define the AutoTester class
class AutoTester(Agent):
    def __init__(self, client=None, solver="gurobipy", **kwargs):
        super(AutoTester, self).__init__(
            name="AutoTester",
            description="This is a testing agent that double checks the validity of the solution",
            client=client,
            **kwargs
        )
        self.solver = solver

    def generate_reply(self, task: str, state: Dict, sender: "Agent") -> Tuple[str, Dict]:
        print("- Testing agent is called! ")

        # Example of how to use the test_code_template
        constraint = "x + y <= 10"
        variable = "x, y"
        prompt = test_code_template.format(CONSTRAINT=constraint, VARIABLE=variable)

        response = self.llm_call(prompt)
        print(response)

        return response, state

# Example usage
if __name__ == "__main__":
    tester = AutoTester()
    task = "Verify the constraints"
    state = {}
    sender = None
    tester.generate_reply(task, state, sender)

- Testing agent is called! 


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



You are in a team of optimization experts, and you are responsible for testing the validity of the solution
    obtained for the problem. Your job is to verify that all the constraints of the problem are satisfied by the 
    solution. Here is one constraint you need to verify 

    x + y <= 10

    This constraint is built based on the following variables 

    x, y

    Now the problem is solved and solution is obtained. 
    Please write Python code that gets values from the Python variables and then verify the constraint.

1. __init__ ( self, parameters, tbody, default_validation_function, __pycall__, __text__ );

2. __init__ ( self, parameters, tbody, default_validation_function, __pycall__ );

3. __pycall__ ( "cpy" );

4. __init__ ( self, parameters, tbody, default_validation_function, __pycall__ );

5. __init__ ( self, parameters, tbody, default_validation_function, __pycall__ );

6. self._frozen. __init__ ( "cpy" );

7. py. __set_initializer ( self, self. parameters );

8. sel

In [3]:
# Install necessary packages
!pip install transformers matplotlib pandas

# Import necessary libraries
from typing import Dict, Tuple
import json
from transformers import pipeline
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import sys

# Define the test code template
test_code_template = """
You are in a team of optimization experts, and you are responsible for testing the validity of the solution
    obtained for the problem. Your job is to verify that all the constraints of the problem are satisfied by the
    solution. Here is one constraint you need to verify

    {CONSTRAINT}

    This constraint is built based on the following variables

    {VARIABLE}

    Now the problem is solved and solution is obtained.
    Please write Python code that gets values from the Python variables and then verify the constraint.
"""

# Define the Agent class
class Agent:
    def __init__(self, name, description, client=None, llm="gpt2", **kwargs):
        self.name = name
        self.description = description
        self.client = client
        self.system_prompt = "You're a helpful assistant."
        self.kwargs = kwargs
        self.llm = llm

    def llm_call(self, prompt: str) -> str:
        generator = pipeline('text-generation', model=self.llm)
        response = generator(prompt, max_length=500, num_return_sequences=1)
        return response[0]['generated_text']

    def generate_reply(self, task: str, state: Dict, sender: "Agent") -> Tuple[str, Dict]:
        return ("This is a reply from the agent. REPLY NOT IMPLEMENTED! Terminate the whole process!", state)

# Define the AutoTester class
class AutoTester(Agent):
    def __init__(self, client=None, solver="gurobipy", **kwargs):
        super(AutoTester, self).__init__(
            name="AutoTester",
            description="This is a testing agent that double checks the validity of the solution",
            client=client,
            **kwargs
        )
        self.solver = solver

    def generate_reply(self, task: str, state: Dict, sender: "Agent") -> Tuple[str, Dict]:
        print("- Testing agent is called! ")

        # Example of how to use the test_code_template
        constraint = "x + y <= 10"
        variable = "x, y"
        prompt = test_code_template.format(CONSTRAINT=constraint, VARIABLE=variable)

        response = self.llm_call(prompt)
        print(response)

        return response, state

# Example usage
if __name__ == "__main__":
    tester = AutoTester()
    task = "Verify the constraints"
    state = {}
    sender = None
    tester.generate_reply(task, state, sender)

# Additional functionality
class CustomClass:
    def __init__(self, parameters, tbody, default_validation_function, __pycall__, __text__=None):
        self.parameters = parameters
        self.tbody = tbody
        self.default_validation_function = default_validation_function
        self.__pycall__ = __pycall__
        self.__text__ = __text__

    def __pycall__(self, method_name):
        print(f"Calling method: {method_name}")

    def _frozen_init(self, value):
        self.__pycall__("cpy")
        self._set_initializer(self.parameters)

    def _set_initializer(self, parameters):
        self.parameters = parameters

    def _get_property_by_type(self, parameters, x_func, y_func, z_func=None):
        x = x_func(parameters)
        y = y_func(parameters)
        z = z_func(parameters) if z_func else None
        return f"x: {x}, y: {y}, z: {z}"

    def _unmarshal(self, value):
        self.__pycall__("cpy")

    def get_property_by_type(self, parameters, x_func, y_func):
        x = x_func(parameters)
        y = y_func(parameters)
        return f"x: {x}, y: {y}"

# Example usage of CustomClass
if __name__ == "__main__":
    custom_obj = CustomClass(parameters="params", tbody="tbody", default_validation_function=lambda x: x, __pycall__=lambda x: x)
    custom_obj.__pycall__("example_method")
    custom_obj._frozen_init("cpy")
    print(custom_obj.get_property_by_type("params", lambda x: x + "x", lambda y: y + "y"))

- Testing agent is called! 


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



You are in a team of optimization experts, and you are responsible for testing the validity of the solution
    obtained for the problem. Your job is to verify that all the constraints of the problem are satisfied by the 
    solution. Here is one constraint you need to verify 

    x + y <= 10

    This constraint is built based on the following variables 

    x, y

    Now the problem is solved and solution is obtained. 
    Please write Python code that gets values from the Python variables and then verify the constraint.

Now, in the second step we will verify the constraint. Let´s write a program that checks the variables before building on it (again, as stated above) and calculates the constraints:
x: paramsx, y: paramsy
